# Google Colaboratoryを使ってESM/ESM-RISM計算を行う

In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

## Quantum ESPRESSOをインストールする。

In [ ]:
!git clone https://github.com/minoru-otani/q-e.git
!apt-get install -y libfftw3-3 libfftw3-dev libfftw3-doc
%cd q-e
!DFLAGS='-D__FFTW3 -D__MPI' FFT_LIBS='-lfftw3' ./configure
!make pw pp

### パスを設定

In [ ]:
import os
os.environ['PATH'] = "/content/q-e/bin:"+os.environ['PATH']

## ESM/ESM-RISMに対応したASEをインストールする

In [ ]:
import importlib
if importlib.util.find_spec("ase") is None:
    !pip install git+https://github.com/minoru-otani/qe_tutorial_JP.git@qe_rism

インポートする

In [2]:
import matplotlib.pyplot as plt
from ase import Atoms
from ase.io import read, write
from ase.visualize import view
from ase.visualize.plot import plot_atoms
from ase.constraints import FixAtoms
from ase.io.cube import read_cube_data
from ase.build import fcc100, add_adsorbate, molecule
from ase.geometry import wrap_positions
from ase.calculators.espresso import Espresso

## スラブモデルでESM計算を実施する

### 擬ポテンシャルを取ってくる。

In [ ]:
!mkdir /content/calc
%cd /content/calc
!wget https://www.quantum-espresso.org/upf_files/Al.pbe-n-van.UPF

### Al(100)のスラブモデルを作成する。

ASEを使ってスラブモデルを作成する。

In [ ]:
slab = fcc100('Al', size=(1,1,4), vacuum=10.0)
slab.wrap() # wrap extended atoms into a unit cell.
fig, ax = plt.subplots(2,1, figsize=(12, 6))
ax[0].set_axis_off()
ax[1].set_axis_off()
plot_atoms(slab, ax[0], radii=1.0, rotation=('0x,0y,0z'))
plot_atoms(slab, ax[1], radii=1.0, rotation=('90x,90y,90z'))

ESM計算用にスラブの位置をずらす

In [ ]:
lvac = 10.0
rvac = 10.0
vac = (lvac + rvac)/2.0
slab = fcc100('Al', size=(1,1,4), vacuum=vac)
slab.wrap() # wrap extended atoms into a unit cell.
slab.translate((0.0,0.0,lvac - vac)) # shift atoms to have given vacuum thickness
slab.translate((0.0,0.0,-slab.cell[2,2]/2.0)) # shift atoms to fit ESM/ESM-RISM model
fig, ax = plt.subplots(1,1, figsize=(12, 6))
ax.set_axis_off()
plot_atoms(slab, ax, radii=1.0, rotation=('90x,90y,90z'))

### 計算を実行する。

In [ ]:
pseudopotentials = {'Al':'Al.pbe-n-van.UPF'}
input_data = {
    'control': {
        'calculation': 'scf',
        'restart_mode': 'from_scratch',
        'prefix': 'Al100_bc2',
        'disk_io': 'low'
    },
    'system': {
        'ibrav': 0,
        'ecutwfc': 20,
        'ecutrho': 160,
        'occupations' : 'smearing',
        'smearing':'mp',
        'degauss' : 0.03,
        'assume_isolated': 'esm',
        'esm_bc': 'bc1'
    },
    'electrons': {
        'mixing_beta': 0.3
    }
}

calc = Espresso(pseudo_dir='/content/calc/',
                pseudopotentials=pseudopotentials,
                kpts=(16, 16, 1),
                koffset=(1, 1, 0),
                input_data=input_data)
slab.set_calculator(calc)

slab.get_potential_energy()
fermi_level = calc.get_fermi_level()
print(fermi_level)

###　結果を解析する

esm1に出力されたデータをプロットする。

In [2]:
plot_esm1('Al100_bc2.esm1',titlename='Al100 bc1')

#!/usr/bin/env python3
# -*- coding: utf-8 -*-
import matplotlib.pyplot as plt
import numpy as np
import math


def plot_esm1(filename, titlename=''):
    esm1 = np.loadtxt(filename, comments='#')

    fig = plt.figure(
        figsize=(6, 4),  # inch
#       dpi=100,  # dpi
#        edgecolor='black',
#        linewidth='1'
    )

    fig.subplots_adjust(wspace=0.5, hspace=0.5)
    fig.suptitle(titlename)

    ax1 = fig.add_subplot(221)
    ax2 = fig.add_subplot(222)
    ax3 = fig.add_subplot(223)
    ax4 = fig.add_subplot(224)

    ax1.set_xlabel('z (A)')
    ax1.set_ylabel('rho (e/A)')
    ax2.set_xlabel('z (A)')
    ax2.set_ylabel('V_hartree (eV)')
    ax3.set_xlabel('z (A)')
    ax3.set_ylabel('V_ion (eV)')
    ax4.set_xlabel('z (A)')
    ax4.set_ylabel('V_electrostatic (eV)')

    ax4.axhline(0.0, linewidth=1, linestyle='dashed', color='black')

    ax1.plot(esm1[:, 0], esm1[:, 1], color='black', linestyle='solid')
    ax2.plot(esm1[:, 0], esm1[:, 2], color='black', linestyle='solid')
    ax3.plot(esm1[:, 0], esm1[:, 3], color='black', linestyle='solid')
    ax4.plot(esm1[:, 0], esm1[:, 4], color='black', linestyle='solid')

    plt.show()


def plot_1drism(filename, titlename='', normalization=False, max_x=None):
    with open(filename, 'r') as file:
        data_1drism = file.readlines()
        # line_molecules = data_1drism[3].split()
        line_atoms = data_1drism[4].split()

    data_1drism = np.loadtxt(filename, comments='#', skiprows=5)

    number_of_sublines = int(math.ceil((len(line_atoms) - 1) / 3))
    number_of_subplots = len(line_atoms) - 1

    fig = plt.figure(
        figsize=(15, 2 * number_of_sublines),  # inch
#        dpi=100,  # dpi
#        edgecolor='black',
#        linewidth=1
    )

    fig.subplots_adjust(wspace=0.5, hspace=0.75)
    fig.suptitle(titlename)

    factor_norm = 1

    for n_plot in range(1, number_of_subplots + 1):
        if normalization:
            factor_norm = data_1drism[-1, n_plot]

        ax1 = fig.add_subplot(number_of_sublines, 3, n_plot)
        if max_x is not None:
            ax1.set_xlim([0, max_x])

        ax1.set_xlabel('r (A)')
        ax1.set_ylabel('rdf')

        if (number_of_subplots % 3) == 0:
            if n_plot <= (number_of_subplots // 3 - 1) * 3:
                ax1.set_xlabel('')
            # ax1.set_xticklabels([])

        ax1.set_title(line_atoms[n_plot])
        ax1.plot(data_1drism[:, 0], data_1drism[:, n_plot] / factor_norm, color='black', linestyle='solid')

    #    plt.savefig('%s.pdf'%titlename)
    plt.show()


def plot_rism1(filename, titlename=''):
    with open(filename, 'r') as file:
        data_rism1 = file.readlines()
        line_data = data_rism1[1].split()

    data_rism1 = np.loadtxt(filename, comments='#', skiprows=2)

    line_data = line_data[1:]
    new_line_data = [line_data[0] + ' ' + line_data[1]]
    for i in range(2, len(line_data), 3):
        new_line_data.append(line_data[i] + ' ' + line_data[i + 1] + ' ' + line_data[i + 2])

    number_of_sublines = int(math.ceil((len(new_line_data) - 1) / 3))
    number_of_subplots = len(new_line_data) - 1

    fig = plt.figure(
        figsize=(15, 2 * number_of_sublines),  # inch
#        dpi=100,  # dpi
#       edgecolor='black',
#        linewidth=1
    )

    fig.subplots_adjust(wspace=0.5, hspace=0.75)
    fig.suptitle(titlename)

    for n_plot in range(1, number_of_subplots + 1):
        ax1 = fig.add_subplot(number_of_sublines, 3, n_plot)
        ax1.set_xlabel(new_line_data[0])
        ax1.set_ylabel(new_line_data[n_plot])
        # ax1.set_title(line_atoms[n_plot])
        ax1.plot(data_rism1[:, 0], data_rism1[:, n_plot], color='black', linestyle='solid')

    plt.show()

ModuleNotFoundError: No module named 'plot_esm1'

### Matplotlibを用いてesm1ファイルのデータをプロットする関数を定義

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
import matplotlib.pyplot as plt
import numpy as np
import math


def plot_esm1(filename, titlename=''):
    esm1 = np.loadtxt(filename, comments='#')

    fig = plt.figure(
        figsize=(6, 4),  # inch
#       dpi=100,  # dpi
#        edgecolor='black',
#        linewidth='1'
    )

    fig.subplots_adjust(wspace=0.5, hspace=0.5)
    fig.suptitle(titlename)

    ax1 = fig.add_subplot(221)
    ax2 = fig.add_subplot(222)
    ax3 = fig.add_subplot(223)
    ax4 = fig.add_subplot(224)

    ax1.set_xlabel('z (A)')
    ax1.set_ylabel('rho (e/A)')
    ax2.set_xlabel('z (A)')
    ax2.set_ylabel('V_hartree (eV)')
    ax3.set_xlabel('z (A)')
    ax3.set_ylabel('V_ion (eV)')
    ax4.set_xlabel('z (A)')
    ax4.set_ylabel('V_electrostatic (eV)')

    ax4.axhline(0.0, linewidth=1, linestyle='dashed', color='black')

    ax1.plot(esm1[:, 0], esm1[:, 1], color='black', linestyle='solid')
    ax2.plot(esm1[:, 0], esm1[:, 2], color='black', linestyle='solid')
    ax3.plot(esm1[:, 0], esm1[:, 3], color='black', linestyle='solid')
    ax4.plot(esm1[:, 0], esm1[:, 4], color='black', linestyle='solid')

    plt.show()


def plot_1drism(filename, titlename='', normalization=False, max_x=None):
    with open(filename, 'r') as file:
        data_1drism = file.readlines()
        # line_molecules = data_1drism[3].split()
        line_atoms = data_1drism[4].split()

    data_1drism = np.loadtxt(filename, comments='#', skiprows=5)

    number_of_sublines = int(math.ceil((len(line_atoms) - 1) / 3))
    number_of_subplots = len(line_atoms) - 1

    fig = plt.figure(
        figsize=(15, 2 * number_of_sublines),  # inch
#        dpi=100,  # dpi
#        edgecolor='black',
#        linewidth=1
    )

    fig.subplots_adjust(wspace=0.5, hspace=0.75)
    fig.suptitle(titlename)

    factor_norm = 1

    for n_plot in range(1, number_of_subplots + 1):
        if normalization:
            factor_norm = data_1drism[-1, n_plot]

        ax1 = fig.add_subplot(number_of_sublines, 3, n_plot)
        if max_x is not None:
            ax1.set_xlim([0, max_x])

        ax1.set_xlabel('r (A)')
        ax1.set_ylabel('rdf')

        if (number_of_subplots % 3) == 0:
            if n_plot <= (number_of_subplots // 3 - 1) * 3:
                ax1.set_xlabel('')
            # ax1.set_xticklabels([])

        ax1.set_title(line_atoms[n_plot])
        ax1.plot(data_1drism[:, 0], data_1drism[:, n_plot] / factor_norm, color='black', linestyle='solid')

    #    plt.savefig('%s.pdf'%titlename)
    plt.show()


def plot_rism1(filename, titlename=''):
    with open(filename, 'r') as file:
        data_rism1 = file.readlines()
        line_data = data_rism1[1].split()

    data_rism1 = np.loadtxt(filename, comments='#', skiprows=2)

    line_data = line_data[1:]
    new_line_data = [line_data[0] + ' ' + line_data[1]]
    for i in range(2, len(line_data), 3):
        new_line_data.append(line_data[i] + ' ' + line_data[i + 1] + ' ' + line_data[i + 2])

    number_of_sublines = int(math.ceil((len(new_line_data) - 1) / 3))
    number_of_subplots = len(new_line_data) - 1

    fig = plt.figure(
        figsize=(15, 2 * number_of_sublines),  # inch
#        dpi=100,  # dpi
#       edgecolor='black',
#        linewidth=1
    )

    fig.subplots_adjust(wspace=0.5, hspace=0.75)
    fig.suptitle(titlename)

    for n_plot in range(1, number_of_subplots + 1):
        ax1 = fig.add_subplot(number_of_sublines, 3, n_plot)
        ax1.set_xlabel(new_line_data[0])
        ax1.set_ylabel(new_line_data[n_plot])
        # ax1.set_title(line_atoms[n_plot])
        ax1.plot(data_rism1[:, 0], data_rism1[:, n_plot], color='black', linestyle='solid')

    plt.show()